In [1]:
import numpy as np
import pandas as pd
import math
import json
import time
from sklearn.model_selection import train_test_split
import scipy.sparse
from scipy.sparse import csr_matrix
import warnings; warnings.simplefilter('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Dataset,Reader
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
def prediction_books(uid):
	model = pickle.load(open('model.sav', 'rb'))
	ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1",warn_bad_lines=False)
	ratings.columns = ['userID', 'ISBN', 'bookRating']
	books = pd.read_csv("BX-Books.csv", sep=";", error_bad_lines=False, encoding="latin-1",warn_bad_lines=False);
	books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL'];
	users = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1",warn_bad_lines=False);
	users.columns = ['userID', 'Location', 'Age'];
	ratings_new = ratings[ratings.ISBN.isin(books.ISBN)]
	ratings_new = ratings_new[ratings.userID.isin(users.userID)]
	ratings_explicit = ratings_new[ratings_new.bookRating != 0]
	counts1 = pd.value_counts(ratings_explicit['userID'])
	ratings_explicit = ratings_explicit[ratings_explicit['userID'].isin(counts1[counts1 >= 500].index)]
	from surprise import Dataset,Reader
	reader = Reader(rating_scale=(1, 10))
	data = Dataset.load_from_df(ratings_explicit[['userID', 'ISBN', 'bookRating']], reader)
	reader = Reader(rating_scale=(1, 10))
	data = Dataset.load_from_df(ratings_explicit[['userID', 'ISBN', 'bookRating']], reader)
	from surprise.model_selection import train_test_split
	trainset, testset = train_test_split(data, test_size=.25,random_state=123)
	test_pred=model.test(testset)
	pred = pd.DataFrame(test_pred)
	pred1 = pred[['uid', 'iid', 'r_ui']]
	books1 = books[['ISBN', 'bookTitle', 'bookAuthor']]
	books1.columns = ['iid', 'bookTitle', 'bookAuthor']
	df = pd.merge(pred1, books1, on='iid')
	return str(uid) + "::" + str(df[df['uid'] == uid][['bookTitle', 'bookAuthor']].head(10))

In [8]:
def beginning_books():
	model = pickle.load(open('model.sav', 'rb'))
	ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1",warn_bad_lines=False)
	ratings.columns = ['userID', 'ISBN', 'bookRating']
	books = pd.read_csv("BX-Books.csv", sep=";", error_bad_lines=False, encoding="latin-1",warn_bad_lines=False);
	books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL'];
	users = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1",warn_bad_lines=False);
	users.columns = ['userID', 'Location', 'Age'];
	ratings_new = ratings[ratings.ISBN.isin(books.ISBN)]
	ratings_new = ratings_new[ratings.userID.isin(users.userID)]
	ratings_explicit = ratings_new[ratings_new.bookRating != 0]
	counts1 = pd.value_counts(ratings_explicit['userID'])
	ratings_explicit = ratings_explicit[ratings_explicit['userID'].isin(counts1[counts1 >= 500].index)]
	pred1 = ratings_explicit
	books1 = books[['ISBN', 'bookTitle', 'bookAuthor']]
	df = pd.merge(pred1, books1, on='ISBN')
	res = []
	for i in range(10):
		uid = np.random.choice(list(np.unique(df['userID'].values)))
		res.append(str(uid) + "::" + str(df[df['userID'] == uid][['bookTitle', 'bookAuthor']].head(10)))
	return res

In [9]:
beginning_books()

["197659::                                         bookTitle        bookAuthor\n97                               The Fourth Estate    Jeffrey Archer\n151                           A Light in the Attic  Shel Silverstein\n307   Dance Hall of the Dead (Joe Leaphorn Novels)    Tony Hillerman\n343                              White Shell Woman     James D. Doss\n456               Charlotte's Web (Trophy Newbery)       E. B. White\n758    A Long Way from Chicago: A Novel in Stories      Richard Peck\n1438                                   Naked Sun 2      Isaac Asimov\n1525                                         Congo  Michael Crichton\n1639                                  The Talisman      STEPHEN KING\n1955                                   Black House      Stephen King",
 "158295::                                              bookTitle           bookAuthor\n125                                         Burn Factor           Kyle Mills\n186                                      Bare Necessi

In [13]:
prediction_books(158295)

'158295::                                             bookTitle      bookAuthor\n6                   Homeopathy for Babies and Children    Trevor Smith\n51   N Is for Noose (Kinsey Millhone Mysteries (Har...     Sue Grafton\n189     Stealing Shadows (Shadows Trilogy (Paperback))      Kay Hooper\n190                                          Flashback  Michael Palmer\n195                                        Burn Factor      Kyle Mills\n226  Tippy-Toe Murder: A Lucy Stone Mystery (Lucy S...    Leslie Meier\n262  The Drawing of the Three (The Dark Tower, Book 2)    Stephen King\n268                                       Dreamcatcher    Stephen King\n312  O Is for Outlaw (Kinsey Millhone Mysteries (Au...     Sue Grafton\n538                                \\O\\" Is for Outlaw"     Sue Grafton'